In [ ]:
import pandas as pd
import unicodedata
import re

In [ ]:
path = "./raw/sentiment.csv"
df = pd.read_csv(path, quotechar='"', on_bad_lines="skip")

In [ ]:
for k in range(10):
    print(df.values[k][3])

In [ ]:
print(df.head())

In [ ]:
def unicode_to_ascii(s):
        return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
def preprocess_sentence(w):
        w = unicode_to_ascii(w.lower().strip())

        # creating a space between a word and the punctuation following it
        # eg: "he is a boy." => "he is a boy ."
        # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
        w = re.sub(r"([?.!,¿])", r" \1 ", w)
        w = re.sub(r'[" "]+', " ", w)

        # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
        w = re.sub(r"[^a-zA-Z?!,¿]+", " ", w)

        w = w.strip()

        # adding a start and an end token to the sentence
        # so that the model know when to start and stop predicting.
        w = '<start> ' + w + ' <end>'
        return w

In [ ]:
w = df['SentimentText'].values[23]
print(w)
w = unicode_to_ascii(w.lower().strip())
w = re.sub(r"([?.!,¿])", r" \1 ", w)
w = re.sub(r'[" "]+', " ", w)
w = re.sub(r"[^a-zA-Z?!,¿]+", " ", w)
w = w.strip()
print(w)

In [ ]:
df['ProcessedText'] = df['SentimentText'].apply(preprocess_sentence)

In [16]:
import pickle

# Load training/testing data
with open('train_valid_test_data.pkl', 'rb') as f:
    X_train, y_train, X_valid, y_valid, X_test, y_test = pickle.load(f)

# Load tokenizer
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

In [21]:
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)
print("====")
print(y_train[:10])
print(y_valid[:10])
print(y_test[:10])

(1262889, 100)
(157861, 100)
(157862, 100)
====
[1, 1, 0, 1, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 0, 1, 0, 1, 1, 1, 0]
[0, 0, 1, 0, 0, 0, 1, 0, 1, 1]


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Sample dataframe
df = pd.DataFrame({
    'features': ['feature'+str(i) for i in range(10)],
    'labels': ['label'+str(i) for i in range(10)],
})

features = df['features']
labels = df['labels']

X_train, X_valid, X_test, y_train, y_valid, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

ValueError: not enough values to unpack (expected 6, got 4)

In [ ]:
from sklearn.model_selection import train_test_split
test_size = 0.1
features = df['ProcessedText']
labels = df['Sentiment']
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=test_size, random_state=42, stratify=labels)
X_train, X_test, y_train, y_test = X_train.tolist(), X_test.tolist(), y_train.tolist(), y_test.tolist()

In [219]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

# Tokenize your text data
num_words = 1000
tokenizer = Tokenizer(num_words=num_words, lower=False, filters='', oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_test = tokenizer.texts_to_sequences(X_test)


In [220]:
# Pad your sequences
max_sequence_length = 100
X_train_padded = pad_sequences(sequences_train, maxlen=max_sequence_length, padding="post")
X_test_padded = pad_sequences(sequences_test, maxlen=max_sequence_length, padding="post")

In [24]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
embedding_dim = 64
num_cells = 10
num_classes = 2
num_words=1000
max_sequence_length=100
model = Sequential()
model.add(Embedding(num_words, embedding_dim, input_length=max_sequence_length))
model.add(LSTM(num_cells))
model.add(Dense(num_classes, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])

2023-06-04 18:02:41.951001: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-04 18:02:41.953370: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-04 18:02:41.954852: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [223]:
print(type(X_train_padded))
print(X_train_padded.shape)

<class 'numpy.ndarray'>
(1420750, 100)


In [25]:
ypred = model(X_train_padded[0:10])
ypred

<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
array([[0.49729493, 0.5084913 ],
       [0.49729493, 0.5084913 ],
       [0.49729493, 0.5084913 ],
       [0.49729493, 0.5084913 ],
       [0.49729493, 0.5084913 ],
       [0.49729493, 0.5084913 ],
       [0.49729493, 0.5084913 ],
       [0.49729493, 0.5084913 ],
       [0.4972949 , 0.5084913 ],
       [0.4972949 , 0.5084913 ]], dtype=float32)>

In [ ]:
for i in range(5):
    print("Original Text:", X_train[i])
    print("Sequence:", sequences_train[i])
    print(type(X_train[i]))
    print(len(X_train[i]))
    print(len(sequences_train[i]))
    print("---")

In [ ]:
print(X_train_padded.shape)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create the transform
vectorizer = TfidfVectorizer()

# Joining the tokens into sentences
X_train_joined = [' '.join(tokens) for tokens in X_train]
X_test_joined = [' '.join(tokens) for tokens in X_test]

X_train_joined = [sentence.replace('& quot ;', '') for sentence in X_train_joined]
X_test_joined = [sentence.replace('& quot ;', '') for sentence in X_test_joined]

# Create the transform
vectorizer = TfidfVectorizer()

# Tokenize and build vocab
vectorizer.fit(X_train_joined)

# summarize
print(vectorizer.vocabulary_)

# encode document
X_train_tfidf = vectorizer.transform(X_train_joined)
X_test_tfidf = vectorizer.transform(X_test_joined)

print(X_train_tfidf.shape)
print(X_test_tfidf.shape)

In [ ]:
print(X_train_joined[:10])
print(X_train.head(10))

In [ ]:
print(type(X_train_tfidf))
print(type(y_train))
print(X_train_tfidf.shape)

In [ ]:
from scipy import sparse
import pickle

sparse.save_npz("./sentiment/X_train_tfidf.npz", X_train_tfidf)
sparse.save_npz("./sentiment/X_test_tfidf.npz", X_test_tfidf)
sparse.save_npz("./sentiment/y_train.npz", y_train)
sparse.save_npz("./sentiment/y_test.npz", y_train)